In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# <span style='color:Orange'> GOAL: prototype </span>
#### Get detailed information about the types of cargo that an operator carries, as well as breakouts of the different vehicle types that operator uses

## <span style='color:Pink'> Read the downloaded csv file </span> from below url
* https://ai.fmcsa.dot.gov/SMS/Tools/Downloads.aspx
* https://ai.fmcsa.dot.gov/SMS/Carrier/21800/CarrierRegistration.aspx
* **Prototype :** Take a small sample dataset of (say) **5 rows** from the downloaded csv file for implementing the <span style='color:Blue'>Assignment</span>

In [2]:
import pandas as pd

url = ".\census\FMCSA_CENSUS1_2022Sep\FMCSA_CENSUS1_2022Sep.csv"
# url = "file_name_5rows.csv"  #Read smaller file for prototype - 5 rows 

#Load the csv file - to scrape the "Carrier Registration Details" page.
data = pd.read_csv(url, encoding= 'unicode_escape')
type(data)

pandas.core.frame.DataFrame

## Analyze the schema of the csv, before parsing

In [3]:
len(data.columns)
data.columns

29

Index(['DOT_NUMBER', 'LEGAL_NAME', 'DBA_NAME', 'CARRIER_OPERATION', 'HM_FLAG',
       'PC_FLAG', 'PHY_STREET', 'PHY_CITY', 'PHY_STATE', 'PHY_ZIP',
       'PHY_COUNTRY', 'MAILING_STREET', 'MAILING_CITY', 'MAILING_STATE',
       'MAILING_ZIP', 'MAILING_COUNTRY', 'TELEPHONE', 'FAX', 'EMAIL_ADDRESS',
       'MCS150_DATE', 'MCS150_MILEAGE', 'MCS150_MILEAGE_YEAR', 'ADD_DATE',
       'OIC_STATE', 'NBR_POWER_UNIT', 'DRIVER_TOTAL', 'RECENT_MILEAGE',
       'RECENT_MILEAGE_YEAR', 'VMT_SOURCE_ID'],
      dtype='object')

In [4]:
data.dtypes

DOT_NUMBER               int64
LEGAL_NAME              object
DBA_NAME                object
CARRIER_OPERATION       object
HM_FLAG                 object
PC_FLAG                 object
PHY_STREET              object
PHY_CITY                object
PHY_STATE               object
PHY_ZIP                 object
PHY_COUNTRY             object
MAILING_STREET          object
MAILING_CITY            object
MAILING_STATE           object
MAILING_ZIP             object
MAILING_COUNTRY         object
TELEPHONE               object
FAX                     object
EMAIL_ADDRESS           object
MCS150_DATE             object
MCS150_MILEAGE         float64
MCS150_MILEAGE_YEAR    float64
ADD_DATE                object
OIC_STATE               object
NBR_POWER_UNIT         float64
DRIVER_TOTAL           float64
RECENT_MILEAGE           int64
RECENT_MILEAGE_YEAR      int64
VMT_SOURCE_ID          float64
dtype: object

## Total number of Rows and Columns

In [5]:
pd.options.display.max_rows = 10
data.shape

(2010116, 29)

## Sample Data

In [6]:
df=data.head(5)
pd.options.display.max_rows = 5
df

,DOT_NUMBER,LEGAL_NAME,DBA_NAME,CARRIER_OPERATION,HM_FLAG,PC_FLAG,PHY_STREET,PHY_CITY,PHY_STATE,PHY_ZIP,...,MCS150_DATE,MCS150_MILEAGE,MCS150_MILEAGE_YEAR,ADD_DATE,OIC_STATE,NBR_POWER_UNIT,DRIVER_TOTAL,RECENT_MILEAGE,RECENT_MILEAGE_YEAR,VMT_SOURCE_ID
0,1,FEDERAL MOTOR CARRIER SAFETY ADMINISTRATION,FMCSA TECHNOLOGY DIVISION,A,N,N,1200 NEW JERSEY AVENUE SE,WASHINGTON,DC,20590,...,23-SEP-21,3000.0,2020.0,01-JUN-74,DC,4.0,3.0,3000,2020,1.0
1,10000,POWELL DISTRIBUTING CO INC,NaN,A,N,N,NaN,NaN,NaN,NaN,...,04-MAR-22,16932.0,2021.0,01-JUN-74,OR,1.0,1.0,16932,2021,1.0
2,1000000,JAMES EARL KILLINGSWORTH JR,JAMES KILLINGSWORTH TRUCKING,A,N,N,15 GASLINE ROAD,PHENIX CITY,AL,36870,...,06-AUG-22,1.0,2020.0,23-JAN-02,AL,1.0,1.0,1,2020,1.0
3,1000002,NEW JERSEY BOOM & ERECTORS INC,NaN,A,N,N,NaN,NaN,NaN,NaN,...,19-JUL-22,42000.0,2021.0,22-JAN-02,PA,1.0,1.0,42000,2021,1.0
4,1000004,RAY TRUCKING LLC,NaN,C,N,N,218 PEACHTREE ST,WARWICK,GA,31796,...,30-APR-22,100.0,2022.0,22-JAN-02,GA,2.0,1.0,100,2022,1.0


##  get the  <span style='color:Blue'>Carrier Details</span> page for each operator

* **note the structure from the URL** https://ai.fmcsa.dot.gov/SMS/Carrier/[DOT_NUMBER]/CarrierRegistration.aspx

In [7]:
def prepareUrl(dotId):
    url = 'https://ai.fmcsa.dot.gov/SMS/Carrier/'+str(dotId)+'/CarrierRegistration.aspx'
    return url

## Web Scraping - API

* Use the <span style='color:Blue'>BeautifulSoup</span> library to parse this document, and extract the text 

In [8]:
import csv
import requests
from bs4 import BeautifulSoup

In [9]:
def getSoupExtract(url):
    HTMLFileToBeOpened = open(url, "r")
    contents = HTMLFileToBeOpened.read()
    beautifulSoupText = BeautifulSoup(contents, 'html.parser')
    return beautifulSoupText

## findCargoNames()
* Extract/Pull the lists of **cargo** carried 
* return cargoVehcleRow -- Row of data associated to one **vehicle Operator**

In [10]:
def findCargoNames(cargoVehcleRow,beautifulSoupText):
    job_elements = beautifulSoupText.find_all("ul", class_="cargo")
    for job_element in job_elements:
        cargos = ""
        checked_cargo_elements = job_element.find_all("li", class_="checked")
        for cargo_element in checked_cargo_elements:
            cargo_name = cargo_element.find(text=True, recursive=False)
            if not cargos:
                cargos = "'"+cargo_name+"'"
            else:
                cargos = cargos +" | " +"'"+cargo_name+"'"

#         print(cargos)
        cargoVehcleRow.append(cargos)

## findVehicletype()
* Extract/Pull the lists of the **vehicle type** carried 
* return cargoVehcleRow -- Row of data associated to one **vehicle Operator**

In [11]:
def findVehicletype(cargoVehcleRow,beautifulSoupText):
    veh_elements = beautifulSoupText.find_all("th", class_="vehType")
    vehicles = ""
    for veh_element in veh_elements:
        vehicle_elements = veh_element.find_next_siblings("td")
        for vehicle_element in vehicle_elements:
            if(vehicle_element.text == '1'):
                vehicle_name = veh_element.find(text=True, recursive=False)
                vehicle_name = vehicle_name.replace('*','')
                if not vehicles:
                    vehicles = "'"+vehicle_name+"'"
                else:
                    vehicles = vehicles +" | " +"'"+vehicle_name+"'"

#     print(vehicles)
    cargoVehcleRow.append(vehicles)

## writeToCSV()
* Write the output of one Operator into a separate csv file. Example: 1000000.csv
* NOTE: Syntax: <span style='color:Pink'>DOT_NUMBER.csv</span>
* Write output: <span style='color:Blue'> ONE csv per DOT_NUMBER</span>

In [12]:
def writeToCSV(cargoVehcleRow,filename):
    rows = []
    rows.append(cargoVehcleRow)

    fields = ['CARGO_CARRIED', 'VEHICLE_TYPE']
    
    # writing to csv file
    with open(filename, 'w',newline='') as csvfile:
        # create a csv writer object
        csvwriter = csv.writer(csvfile)

        # Write the fields/Header into CSV
        csvwriter.writerow(fields)

        # Write the data rows into CSV
        csvwriter.writerows(rows)

## extractDataAndWriteToCSVFile()
* <span style='color:Blue'>The Driver method</span>

In [13]:

def extractDataAndWriteToCSVFile(url,filename):
    # print(url)
    
    # Reading from website link
    page = requests.get(url)
    beautifulSoupText = BeautifulSoup(page.content, "html.parser")
    
    # Define rows data
    cargoVehcleRow = []
    
    #  Finding Cargo names and preparing comma separated string
    findCargoNames(cargoVehcleRow,beautifulSoupText)
    ##print("1.CargoNames: --------------->",cargoVehcleRow)
    # Finding Vehicle type
    findVehicletype(cargoVehcleRow,beautifulSoupText)
    ##print("2.Vehicletype: --------------->",cargoVehcleRow)
    # Writing into CSV file
    writeToCSV(cargoVehcleRow,filename)
    

### Iterate over every record in dataframe and prepare an output csv file for each Operator

In [14]:
# Iterate over every record in dataframe and prepare a csv file for each Operator
for i in range(len(df)):
    dotId = df.iloc[i, 0]
    url = prepareUrl(dotId)
    dotIdStr = str(dotId)
    extractDataAndWriteToCSVFile(url,dotIdStr+".csv")
    print("CSV created successfully for: "+dotIdStr)

CSV created successfully for: 1
CSV created successfully for: 10000
CSV created successfully for: 1000000
CSV created successfully for: 1000002
CSV created successfully for: 1000004


## Now we have multiple Output CSV
* pull the lists of cargo carried as well as the vehicle type breakout table -- is  <span style='color:Green'>completed Successful</span>